In [2]:
# data reading and const setting
import pandas as pd
import numpy as np
import random
from enum import Enum
import math
import copy

In [16]:
def read_from_excel(filename, n, p, n_d):
    infinity=9999
    firefighter_df = pd.read_excel("data/" + filename + "_firefighter_route.xlsx")
    fire_df = pd.read_excel("data/" + filename + "_fire_route.xlsx") 
    burning_time_df = pd.read_excel("data/" + filename + "_burning_time.xlsx") 
    #bprocessing_time_df = pd.read_excel("data/" + filename + "_processing_time.xlsx") 
    A_p = [[infinity for i in range(n+1)] for j in range(n+1)] #only travel time
    A_f = [[infinity for i in range(n+1)] for j in range(n+1)] #travel time plus burning time
    processing_time = {}
    for k in range(1,len(p)+1):
        processing_time[k]={}
    for i in range(len(fire_df.index)):
        A_f[fire_df.iloc[i]['i']][fire_df.iloc[i]['j']] = fire_df.iloc[i]['travel time']
    for i in range(1, n+1):
        for j in range(1, n+1):
            if A_f[i][j] != infinity:
                A_f[i][j] += burning_time_df.iloc[i-1]['burning time']
            elif i == j:
                A_f[i][j]=0
                A_p[i][j]=0
            
    for i in range(len(firefighter_df.index)):
        A_p[firefighter_df.iloc[i]['i']][firefighter_df.iloc[i]['j']] = firefighter_df.iloc[i]['travel time']
    for i in range(n+1):
        A_p[0][i]=0
        A_p[i][0]=0
    for k in range(1,len(p)+1):
        for i in range(n+1):
            if i in n_d:
                processing_time[k][i] = 0
            elif i == 0:
                processing_time[k][i]=10000
            else:
                processing_time[k][i] = burning_time_df.iloc[i-1]['burning time']*burning_time_df.iloc[i-1]['quantity'] / p[k]
    data = {'A_p': A_p, 'A_f': A_f, 'processing_time': processing_time, 'n': n}
    return data

class FFProblem:
    def __init__(self,data, N_D, N_F, T, P):
        self.M = 1000
        self.A_p = np.array(data['A_p'])
        self.A_p_temp = copy.copy(self.A_p)
        self.A_f = np.array(data['A_f']) 
        self.A_f_temp = copy.copy(self.A_f)
        self.n = data['n']
        self.N_D = N_D
        self.N_F = N_F
        self.processing_time = data['processing_time']
        self.T = T
        self.t = 0
        self.u = [0 for i in range(self.n+1)]
        self.u_bar = [0 for i in range(self.n+1)]
        self.u[N_F[0]]=1
        self.fighter_num=len(P)
    def reset(self):
        self.A_p_temp = copy.copy(self.A_p)
        self.A_f_temp = copy.copy(self.A_f)
        self.t = 0
        self.u = [0 for i in range(self.n+1)]
        self.u_bar = [0 for i in range(self.n+1)]
        self.u[N_F[0]]=1
    def fireDijkstra(self):
        infinity=9999
        dis=np.zeros(self.n+1,dtype=int) #[0,1,2,3,4,...N]
        known=np.zeros(self.n+1,dtype=int)#紀錄是否得到start到該點的最短路徑
        start=self.N_F[0]
        known[start]=1
        for i in range(1,self.n+1):
            dis[i]=self.A_f_temp[start][i]
        #Dijkstra
        for i in range(1,self.n): #將N-1個node的known標記為1
            shortest = infinity
            for j in range(1,N+1):
                if known[j]==0 and dis[j] <= shortest:
                    shortest = dis[j]
                    u=j
            known[u]=1
            for v in range(1,self.n+1):
                if self.A_f_temp[u][v]<infinity:
                    if dis[v] > dis[u]+self.A_f_temp[u][v]:
                        dis[v] = dis[u]+self.A_f_temp[u][v]
        return dis
    def firefighterDijkstra(self,start,k):
        fire_break_out = self.fireDijkstra()
        infinity=9999
        dis=np.zeros(self.n+1,dtype=int) #[0,1,2,3,4,...N]
        known=np.zeros(self.n+1,dtype=int)#紀錄是否得到start到該點的最短路徑
        known[start]=1
        for i in range(1,self.n+1):
            dis[i]=self.A_p_temp[start][i]
        #print(dis)
        for i in range(1,self.n):
            shortest = infinity
            for j in range(1,N+1):
                if known[j]==0 and dis[j] <= shortest:
                    shortest = dis[j]
                    u=j
            known[u]=1
            #print(u,dis)
            if self.t + dis[u] <= fire_break_out[u]: #起始點到該點的最短花費時間要<=該點燒起來的時間才可以去更新shortest path
                for v in range(1,self.n+1):
                    if self.A_p_temp[u][v]<infinity:
                        if dis[v] > dis[u]+self.A_p_temp[u][v]:
                            dis[v] = dis[u]+self.A_p_temp[u][v]
            else:
                dis[u]=-1
        return dis
    def modify_fire_route(self):
        for i in range(1,len(self.u_bar)):
            if self.u_bar[i]==1:
                for j in range(1,self.n+1):
                    if i != j:
                        self.A_f_temp[i][j] = 9999
                        self.A_f_temp[j][i] = 9999
    def modify_fighter_route(self):
        fire_break_out = self.fireDijkstra()
        for i in range(1,self.n+1):
            if fire_break_out[i] < self.t:
                self.u[i]=1
        for i in range(1,len(self.u)):
            if self.u[i]==1:
                for j in range(1,self.n+1):
                    if i != j:
                        self.A_p_temp[i][j]=9999
                        self.A_p_temp[j][i]=9999
    def compute_objective_value(self,protect):
        protect_order = list(protect)
        for i in range(self.n): #check each gene
            if i == 0:
                x = self.N_D[0]
                y = protect_order[i]
                move = self.firefighterDijkstra(x,1)
                #print(x,y)
                #print('can move:',move)
                if move[y] != -1:
                    self.t = self.t + move[y] + self.processing_time[1][y]
                    self.u_bar[y] = 1
                    #將保護的點arc從火焰路徑移除
                    self.modify_fire_route()
                elif move[y] == -1:
                    self.reset()
                    return self.M
            else:
                x,y = protect_order[i-1], protect_order[i]
                #print(x,y)
                move = self.firefighterDijkstra(x,1)
                #print('can move:',move)
                if move[y] != -1:
                    self.t = self.t + move[y] + self.processing_time[1][y]
                    self.u_bar[y] = 1
                    #將保護的點arc從火焰路徑移除
                    self.modify_fire_route()
                elif move[y] == -1:
                    self.reset()
                    return self.M
            #若保護某基因的點後使t>給定的T，則不繼續檢查染色體，直接去算目前保護的點所達成的目標式值
            if self.t > self.T: #choose idle must let t > T
                
                #idle後剩下到T之間會有哪些點燒起來
                last_burn = self.fireDijkstra()
                for i in range(1,self.n+1):
                    if last_burn[i] < self.T:
                        self.u[i]=1
                
                #print(self.u_bar)
                #print(self.A_f_temp)
                break
            else:
                #print(self.u_bar)
                #print(self.A_f_temp)
                #看保護完後的t有哪些點已經燒起來，將燒起來的點從消防員路徑移除
                self.modify_fighter_route()
                #print('t:',self.t)
                #print('-----------------')
        
        #計算損失土地價值
        b = [1 for i in range(self.n+1)]
        total = 0
        for i in range(1,self.n+1):
            total += self.u[i]*b[i]
        self.reset()
        return total
        #print('burn',self.u)
        #print('obj',total)
        
        
        

        
class CrossoverType(Enum):
    PartialMappedCrossover = 1
    OrderCrossover = 2
    PositionBasedCrossover = 3
    
class MutationType(Enum):
    Inversion = 1
    Insertion = 2
    Displacement = 3
    ReciprocalExchange = 4
    
class SelectionType(Enum):
    Deterministic = 1
    Stochastic= 2
    
class GeneticAlgorithm:
    def __init__(self,pop_size,number_of_genes,selection_type,
                 crossover_type,crossover_rate,mutation_type,mutation_rate,
                 compute_objective_value):
        
        self.pop_size = pop_size
        self.selection_type = selection_type
        self.crossover_size = int(pop_size*crossover_rate)
        if(self.crossover_size%2==1):
            self.crossover_size -= 1;
        self.mutation_size =  int(pop_size*mutation_rate)
        self.total_size = self.pop_size+self.mutation_size+self.crossover_size
        self.number_of_genes = number_of_genes
        self.crossover_type = crossover_type
        self.crossover_rate = crossover_rate
        self.mutation_type = mutation_type
        self.mutation_rate = mutation_rate
        self.compute_objective_value = compute_objective_value
        self.least_fitness_factor = 0.3
        self.mapping = [-1 for i in range(self.number_of_genes)] #for crossover
        
    def initialize(self):
        self.selected_chromosomes = np.zeros((self.pop_size,self.number_of_genes))
        self.indexs = np.arange(self.total_size)
        self.chromosomes = np.zeros((self.total_size,self.number_of_genes),dtype=int)
#         self.chromosomes = np.array([[range(1, self.number_of_genes + 1)] * self.total_size], dtype=int)
        for i in range(self.pop_size):
            for j in range(self.number_of_genes):  
                self.chromosomes[i][j] = j
            np.random.shuffle(self.chromosomes[i])
       
        for i in range(self.pop_size,self.total_size):
            for j in range(self.number_of_genes):
#                 self.chromosomes[i][j] = -1
                self.chromosomes[i][j] = j
                
        self.fitness = np.zeros(self.total_size) 
        self.objective_values = np.zeros(self.total_size)
        self.best_chromosome = np.zeros(self.number_of_genes,dtype=int)
        self.best_fitness = 0
        
    def evaluate_fitness(self):
        for i,chromosome in enumerate(self.chromosomes[:self.pop_size]):
            self.objective_values[i] = self.compute_objective_value(chromosome)
           
        min_obj_val = np.min(self.objective_values)
        max_obj_val = np.max(self.objective_values)
        range_obj_val = max_obj_val-min_obj_val
        
        for i,obj in enumerate(self.objective_values):
            self.fitness[i] = max(self.least_fitness_factor*range_obj_val,pow(10,-5))+\
                (max_obj_val-obj)
                
    def update_best_solution(self):
        best_index = np.argmax(self.fitness)
        if(self.best_fitness<self.fitness[best_index]):
            self.best_fitness = self.fitness[best_index]
            for i,gene in enumerate(self.chromosomes[best_index]):
                self.best_chromosome[i] = gene
    
    def shuffle_index(self,length):
        for i in range(length):
            self.indexs[i] = i
        np.random.shuffle(self.indexs[:length])
            
    def perform_crossover_operation(self):
        self.shuffle_index(self.pop_size)
        
        child1_index = self.pop_size
        child2_index = self.pop_size+1
        count_of_crossover = int(self.crossover_size/2)
        for i in range(count_of_crossover):
            parent1_index = self.indexs[i]
            parent2_index  = self.indexs[i+1]
            
            if(self.crossover_type == CrossoverType.PartialMappedCrossover):
                self.partial_mapped_crossover(parent1_index,parent2_index,child1_index,child2_index)
                self.objective_values[child1_index] = self.compute_objective_value(self.chromosomes[child1_index])
                self.objective_values[child2_index] = self.compute_objective_value(self.chromosomes[child2_index])
            
            child1_index +=2
            child2_index +=2
        
    def partial_mapped_crossover(self,p1,p2,c1,c2):
        #reset
        for i in range(self.number_of_genes):
            self.mapping[i] = -1
         
        rand1 = random.randint(0,self.number_of_genes-2)
        rand2 = random.randint(rand1+1,self.number_of_genes-1)
       
        for i in range(rand1,rand2+1):
            c1_gene = self.chromosomes[p2][i] 
            c2_gene = self.chromosomes[p1][i]
            
            if(c1_gene==c2_gene):
                continue
            
            elif(self.mapping[c1_gene]==-1 and self.mapping[c2_gene]==-1):
                self.mapping[c1_gene] = c2_gene
                self.mapping[c2_gene] = c1_gene
                
            elif(self.mapping[c1_gene]==-1):
                self.mapping[c1_gene] =  self.mapping[c2_gene]
                self.mapping[self.mapping[c2_gene]] = c1_gene
                self.mapping[c2_gene] = -2
                
            elif (self.mapping[c2_gene]==-1):
                self.mapping[c2_gene] =  self.mapping[c1_gene]
                self.mapping[self.mapping[c1_gene]] = c2_gene
                self.mapping[c1_gene] = -2
                
            else:
                self.mapping[self.mapping[c1_gene]] = self.mapping[c2_gene]
                self.mapping[self.mapping[c2_gene]] = self.mapping[c1_gene]
                self.mapping[c1_gene] = -3
                self.mapping[c2_gene] = -3
                
        for i in range(self.number_of_genes):
            if(i>=rand1 and i<=rand2):
                self.chromosomes[c1][i] =  self.chromosomes[p2][i]
                self.chromosomes[c2][i] =  self.chromosomes[p1][i]
            else:
                if(self.mapping[self.chromosomes[p1][i]] >=0):
                    self.chromosomes[c1][i] = self.mapping[self.chromosomes[p1][i]]
                else:
                    self.chromosomes[c1][i] =self.chromosomes[p1][i]        
                
                if(self.mapping[self.chromosomes[p2][i]] >=0):
                    self.chromosomes[c2][i] = self.mapping[self.chromosomes[p2][i]]
                else:
                    self.chromosomes[c2][i] =self.chromosomes[p2][i]
        
    def do_roulette_wheel_selection(self,fitness_list):
        sum_fitness = sum(fitness_list)
        transition_probability = [fitness/sum_fitness for fitness in fitness_list]
        
        rand = random.random()
        sum_prob = 0
        for i,prob in enumerate(transition_probability):
            sum_prob += prob
            if(sum_prob>=rand):
               return i
        
    def perform_selection(self):
        if self.selection_type == SelectionType.Deterministic:
            index = np.argsort(self.fitness)[::-1]
        
        elif self.selection_type == SelectionType.Stochastic:
            index = [self.do_roulette_wheel_selection(self.fitness) for i in range(self.pop_size)]
        
        else:
            index = self.shuffle_index(self.total_size)
        
        for i in range(self.pop_size):
            for j in range(self.number_of_genes):
                self.selected_chromosomes[i][j] =  self.chromosomes[index[i]][j]
        
        for i in range(self.pop_size):
            for j in range(self.number_of_genes):
                self.chromosomes[i][j] = self.selected_chromosomes[i][j]
                
    def perform_mutation_operation(self):
        self.shuffle_index(self.pop_size+self.crossover_size)
        child1_index = self.pop_size+self.crossover_size
        for i in range(self.mutation_size):
            if(self.mutation_type==MutationType.Inversion):
                parent1_index = self.indexs[i]
                self.inversion_mutation(parent1_index,child1_index)
                self.objective_values[child1_index] = self.compute_objective_value(self.chromosomes[child1_index])
                child1_index += 1
            
    def inversion_mutation(self,p1,c1):
        rand1 = random.randint(0,self.number_of_genes-2)
        rand2 = random.randint(rand1+1,self.number_of_genes-1)
        for i in range(self.number_of_genes):
            if(i<rand1 or i>rand2):
                self.chromosomes[c1][i] = self.chromosomes[p1][i]
            else:
                index = rand2-(i-rand1)
                self.chromosomes[c1][i] = self.chromosomes[p1][index]

In [25]:
N=6
T=15
N_D=[6]
N_F=[1]
P={1:2}
# order=[0,1,1,2,4,5]
data = read_from_excel("G6", N, P, N_D)
ff = FFProblem(data, N_D, N_F, T, P)

pop_size = 50
# pop_size = 20
selection_type = SelectionType.Deterministic
crossover_type = CrossoverType.PartialMappedCrossover
crossover_rate = 0.2
mutation_type = MutationType.Inversion
mutation_rate = 0.1
solver = GeneticAlgorithm(pop_size,ff.n ,selection_type,
                          crossover_type,crossover_rate,
                          mutation_type,mutation_rate,
                          ff.compute_objective_value)
solver.initialize()

# print(f"{solver.best_chromosome}")

for i in range(100):
    solver.perform_crossover_operation()
    solver.perform_mutation_operation()
    solver.evaluate_fitness()
    solver.update_best_solution()
    solver.perform_selection()
    if(i %10 ==0):
        print(F"iteration {i} :")
        print(f"{solver.best_chromosome}: {ff.compute_objective_value(solver.best_chromosome)}")
        
        
#print('value',ff.compute_objective_value(order))
# print(ff.fireDijkstra())
# ff.firefighterDijkstra(5,1)
# print(ff.A_p)
# print(ff.A_f)

iteration 0 :
[4 0 3 2 1 5]: 3
iteration 10 :
[3 4 0 2 5 1]: 2
iteration 20 :
[3 4 0 2 5 1]: 2
iteration 30 :
[3 4 0 2 5 1]: 2
iteration 40 :
[3 4 0 2 5 1]: 2
iteration 50 :
[3 4 0 2 5 1]: 2
iteration 60 :
[3 4 0 2 5 1]: 2
iteration 70 :
[3 4 0 2 5 1]: 2
iteration 80 :
[3 4 0 2 5 1]: 2
iteration 90 :
[3 4 0 2 5 1]: 2
